In [4]:
import os

In [6]:
!pwd

/home/jovyan/work/galvanize/capstone1/src


In [7]:
for filename in os.listdir('/home/jovyan/work/galvanize/capstone1/src'):
    print(filename)

.DS_Store
.ipynb_checkpoints
NSDUH_data_exploration_ARCHIVED.ipynb
brainstorm.txt
open-data
soccer_data_exploration.ipynb
variables.txt
